In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import re

In [3]:
#He implementado try:, except: en la obtención de cada categoría, consejo de Dmitry
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "N/A"
                        detalles[label] = value
        except:
            detalles = {}

        # Aquí vamos a extraer el Código Postal e Identificador mediante la URL de cada inmueble
        try:
            codigo_postal_match = re.search(r'(\d{5})-', url_inmueble)
            codigo_postal = codigo_postal_match.group(1) if codigo_postal_match else 'NaN'
        except Exception as e:
            codigo_postal = f"Error al extraer código postal: {e}"
        
        try:
            identificador_match = re.search(r'-(\d+_\d+)', url_inmueble)
            identificador = identificador_match.group(1) if identificador_match else 'NaN'
        except Exception as e:
            identificador = f"Error al extraer identificador: {e}"
        
        # Aquí extraemos el timestamp de cada extracción de datos, que nos será util a la hora de actualizar la db
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
def obtener_inmuebles_paginas(base_url, paginas=2):
    todos_los_inmuebles = []
    columnas = set()

    for i in range(1, paginas + 1):
        print(f"Procesando nueva página")
        url_pagina = f"{base_url}{i}/"
        
        try:
            response = requests.get(url_pagina)
            soup = BeautifulSoup(response.text, "html.parser")
            
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp = obtener_detalles_inmueble(href)
                
                ubicacion = base_url.split('/')[-2].split('-')[-1]
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "ubicacion": ubicacion,
                    "codigo_postal": codigo_postal,
                    "identificador": identificador,
                    "timestamp": timestamp
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
            
        except Exception as e:
            print(f"Error al procesar la página {i}: {e}")
        
        time.sleep(2)
    
    df_alquileres = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_alquileres

In [5]:
# Aquí vamos a hacer un diccionario con las URLs de las zonas de los inmuebles que queremos obtener, 
# con la cantidad de paginas correspondientes
urls_zonas_paginas = {
    "https://www.pisos.com/alquiler/pisos-zaragoza/": 1,
    "https://www.pisos.com/alquiler/pisos-huesca/": 1,
    "https://www.pisos.com/alquiler/pisos-teruel/": 1
}

# Aquí creo una función que recorra cada una de las paginas de las URLs del diccionario 
def obtener_inmuebles_varias_zonas(urls_zonas_paginas):
    todos_los_inmuebles = []
    columnas = set()
    
    for base_url, paginas in urls_zonas_paginas.items():
        inmuebles_zona = obtener_inmuebles_paginas(base_url, paginas)
        todos_los_inmuebles.extend(inmuebles_zona.to_dict('records'))
        if not inmuebles_zona.empty:
            columnas.update(inmuebles_zona.columns)
    
    df_alquileres = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_alquileres

In [6]:

df_alquileres = obtener_inmuebles_varias_zonas(urls_zonas_paginas)

Procesando nueva página
Procesando nueva página
Procesando nueva página


In [7]:
df_alquileres

,Adaptado a personas con movilidad reducida,Tipo suelo,Armarios empotrados,ubicacion,Habitaciones,Orientación,Piscina,precio_m2,consumo,Calle alumbrada,...,Planta,agencia,Superficie útil,timestamp,Conservación,Luz,Garaje,Sistema de seguridad,Interior,Cocina equipada
0,NaN,Terrazo,NaN,zaragoza,2,NaN,NaN,13 €/m²,Error al procesar consumo,NaN,...,2ª,FINCAS MUÑOZ,55 m²,2025-02-20 22:42:42,NaN,NaN,1,NaN,NaN,NaN
1,NaN,Gres,Más de 2,zaragoza,4,Sureste,Comunitaria,11 €/m²,"A, A",NaN,...,4ª,FINCAS ALLUSE,133 m²,2025-02-20 22:42:42,A estrenar,NaN,1,NaN,NaN,Completa
2,N/A,Tarima flotante,1,zaragoza,4,Norte,NaN,10 €/m²,"E, E",NaN,...,2ª,COSOCASA,82 m²,2025-02-20 22:42:42,En buen estado,NaN,1,NaN,NaN,N/A
3,NaN,Suelo porcelanico imitacion a madera,Más de 2,zaragoza,3,NaN,Comunitaria,12 €/m²,Error al procesar consumo,NaN,...,4ª,IBERCOIN 2002 S.L,100 m²,2025-02-20 22:42:42,A estrenar,NaN,1,NaN,NaN,N/A
4,N/A,NaN,1,zaragoza,3,NaN,NaN,8 €/m²,Error al procesar consumo,NaN,...,6ª,PUBLIPISOS,100 m²,2025-02-20 22:42:43,En buen estado,NaN,NaN,N/A,NaN,Cocina equipada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,NaN,Gres,1,teruel,2,NaN,NaN,8 €/m²,Error al procesar consumo,NaN,...,2ª,PROPULSALIA GESTIÓN INTEGRAL DE PROPIEDADES,70 m²,2025-02-20 22:43:04,En buen estado,NaN,1,N/A,NaN,Cocina amueblada
61,NaN,Parquet,NaN,teruel,4,NaN,NaN,8 €/m²,Error al procesar consumo,NaN,...,3ª,TASACION Y GESTION INMOBILIARIA L. BONILLO,108 m²,2025-02-20 22:43:04,Reformado,NaN,NaN,NaN,NaN,Cocina amueblada
62,NaN,NaN,NaN,teruel,3,Este,NaN,1 €/m²,Error al procesar consumo,N/A,...,NaN,Agencia no disponible,NaN,2025-02-20 22:43:04,En buen estado,NaN,Más de 2,NaN,NaN,N/A
63,NaN,NaN,NaN,teruel,3,NaN,NaN,7 €/m²,Error al procesar consumo,NaN,...,2ª,Agencia no disponible,NaN,2025-02-20 22:43:05,Reformado,NaN,NaN,NaN,NaN,NaN


In [20]:
df_alquileres.shape

(91, 60)

In [18]:
df_alquileres.columns

Index(['Jardín', 'Amueblado', 'Alcantarillado', 'Orientación', 'emisiones',
       'Planta', 'Exterior', 'consumo', 'No se aceptan mascotas',
       'Conservación', 'Tipo suelo', 'codigo_postal', 'precio', 'Interior',
       'Superficie útil', 'Sistema de seguridad', 'Se aceptan mascotas',
       'Referencia', 'Terraza', 'Teléfono', 'Tipo de casa',
       'Armarios empotrados', 'Trastero', 'identificador', 'superficie',
       'Aire acondicionado', 'Cocina equipada', 'Habitaciones', 'Calefacción',
       'Baños', 'Puerta blindada', 'Superficie solar', 'Ascensor', 'Garaje',
       'nombre', 'href', 'Carpintería interior', 'Chimenea', 'Agua',
       'Portero automático', 'Gas', 'Superficie construida', 'agencia',
       'Urbanizado', 'Calle asfaltada', 'ubicacion', 'Balcón', 'Piscina',
       'Soleado', 'actualizacion', 'Carpintería exterior', 'Vidrios dobles',
       'precio_m2', 'Gastos de comunidad', 'Lavadero', 'Calle alumbrada',
       'Antigüedad', 'timestamp', 'Adaptado a personas

In [14]:
#Guardamos el df resultante en un .csv
df_alquileres.to_csv('alqileres_scrap_completo.csv', index=False)